In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta

url = "https://data-api.cryptocompare.com/futures/v1/latest/tick?market=bybit&instruments=ETH-USD-INVERSE-PERPETUAL"
api_key = "fb1c1304ec08f8514b98c72e3161a8bdba361313a9f0034682c4a321e975189c"

headers = {
    "Authorization": f"Apikey {api_key}"
}

start_date = datetime(2024, 10, 6)
end_date = datetime.now()

current_date = start_date
all_data = []

while current_date <= end_date:
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        data['date'] = current_date.strftime('%Y-%m-%d')
        all_data.append(data)
    else:
        print(f"Error on {current_date.strftime('%Y-%m-%d')}: {response.status_code}")

    current_date += timedelta(days=1)

# Convert list of dictionaries to DataFrame
df = pd.DataFrame(all_data)

# Write to CSV
df.to_csv("crypto_data_eth.csv", index=False)

In [2]:
import pandas as pd
import json

# Read the CSV file
df = pd.read_csv("crypto_data_eth.csv")

# Parse the JSON data
df['Data'] = df['Data'].apply(lambda x: json.loads(x.replace("'", "\"")))

# Normalize the JSON data into a flat table
parsed_data = pd.json_normalize(df['Data'])

# Concatenate the parsed data with the original DataFrame
final_df = pd.concat([df.drop(columns=['Data']), parsed_data], axis=1)

# Write the parsed data to a new CSV file
final_df.to_csv("final_eth.csv", index=False)